In [46]:
import argparse
import math
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import *
import configparser
#import grequests # for async requests, conflicts with requests somehow
import requests
import numpy as np
from numpy import genfromtxt
import h5py
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
import blosc
from IPython.core.debugger import set_trace
import sys
import os
import itertools
from functools import partial
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import cpu_count
import csv
import datetime
import morton

## JLP user python library
import toolbox

In [47]:
gm0 = toolbox.gaussian3dMask([0,0,0], [0.4,0.4,0.4], 11,11,11, [5,5,5])

In [48]:
def F0(cubes):
    f0 = [[np.sum(cubes[i,j,:,:,:]) for j in range(cubes.shape[1])] for i in range(cubes.shape[0])]
    #f0 = np.sum(cube)
    return(f0)
    
def F1(cubes):
    s = cubes.shape

    f1 = [[np.sum(cubes[i,j,:,:,:]) for j in range(cubes.shape[1])] for i in range(cubes.shape[0])]
    out = f1

    return(out)



In [49]:
def getCube(di):
    data = di['rem'].get_cutout(di['ch_rsc'], di['res'], di['xrng'],
          di['yrng'] ,di['zrng'])
    print(di['loc']) #DEBUG
    sys.stdout.flush() #DEBUG
    ### OUTPUT will be a numpy array in Z Y X order 
    #avg = np.mean(data)
    out = np.mean(np.multiply(gm0, data))
    return(out)


In [50]:
CONFIG_FILE = "neurodata.conf"
COLL_NAME = "collman"
EXP_NAME = "M247514_Rorb_1_Site3Align2_LENS_Session1_CROP"
COORD_FRAME = COLL_NAME + "_" + EXP_NAME
BF = 5

LOCATIONS = "rorb_locations_at_buffered.csv"
LOCATIONS = "GoogleDocData.csv"


CHAN_NAMES = ['DAPI1', 'DAPI2', 'DAPI3', 'GABA', 'GAD2', 'Gephyrin', 'GluN1',
                'MBP', 'PSD95', 'synapsin', 'TdTomato', 'VGlut1']

num_threads = 4

In [51]:
config = configparser.ConfigParser()
config.read(CONFIG_FILE)
TOKEN = config['Default']['token']
boss_url = ''.join( ( config['Default']['protocol'],'://',config['Default']['host'],'/v1/' ) )
#print(boss_url)
#'https://api.boss.neurodata.io/v1/'

#intern
rem = BossRemote(CONFIG_FILE)

In [52]:
L = genfromtxt(LOCATIONS,  delimiter=',', skip_header = 1, dtype='int32').tolist()
L

[[217, 204, 30, 0, 0, 0, 0, 0],
 [234, 180, 42, 4, 0, 1, 1, 0],
 [240, 241, 10, 5, 0, 1, 1, 0],
 [201, 211, 12, 6, 0, 0, -1, 1],
 [249, 221, 11, 7, 0, 0, 0, 0],
 [256, 218, 18, 8, 0, 0, 0, 0],
 [228, 183, 39, 9, 0, 0, 1, 0],
 [222, 181, 41, 10, 0, 0, 2, 0],
 [221, 184, 34, 11, 0, 0, 1, 0],
 [187, 192, 34, 12, 0, 0, 0, 0],
 [191, 190, 32, 13, 0, 0, 0, 0],
 [200, 193, 34, 14, 0, 0, 0, 0],
 [203, 194, 23, 15, 0, 0, 0, 0],
 [206, 179, 20, 16, 0, 1, 0, 0],
 [205, 188, 16, 17, 0, 0, 0, 0],
 [209, 188, 20, 18, 0, 0, 0, 0],
 [197, 194, 20, 19, 1, 0, 0, 0],
 [197, 194, 18, 20, 0, 0, 0, 0],
 [194, 182, 18, 21, 0, 1, 0, 0],
 [200, 187, 22, 22, 0, 0, 0, 0],
 [222, 183, 24, 23, 0, 0, 2, 0],
 [230, 177, 33, 24, 0, 0, 0, 0],
 [225, 172, 36, 25, 0, 1, 0, 0],
 [222, 163, 38, 26, 0, 1, 0, 0],
 [207, 164, 41, 27, 0, 1, 0, 0],
 [201, 163, 19, 29, 0, 0, 0, 0],
 [208, 163, 34, 31, 0, 1, 1, 0],
 [210, 167, 6, 32, 1, 1, 0, 0],
 [217, 189, 14, 34, 0, 0, 1, 0],
 [231, 195, 10, 35, 0, 0, 0, 0],
 [225, 178, 14, 3

In [53]:
L = genfromtxt(LOCATIONS,  delimiter=',', skip_header = 1, dtype='int32').tolist()
m = morton.Morton(dimensions=3, bits=64)
Lzo = [m.pack(L[i][0], L[i][1], L[i][2]) for i in range(len(L))]
#Lzo = sorted([m.pack(L[i][0], L[i][1], L[i][2]) for i in range(len(L))])
#                                                                                               
ind = np.argsort(Lzo)
loc = [m.unpack(l) for l in sorted(Lzo)]
                                                                                               
x_rng = [[x[0]-BF, x[0]+BF+1] for x in loc] 
y_rng = [[y[1]-BF, y[1]+BF+1] for y in loc] 
z_rng = [[z[2]-BF, z[2]+BF+1] for z in loc]
ids   = [L[i][3] for i in ind]


In [54]:
ChanList = []
for ch in CHAN_NAMES:
    di = [{
          'rem': rem,
          'ch_rsc':
            ChannelResource(ch,COLL_NAME,EXP_NAME,'image',datatype='uint8'),
          'ch'  : ch,
          'res' : 0,
          'loc' : loc[i],
          'xrng': x_rng[i],  
          'yrng': y_rng[i],  
          'zrng': z_rng[i],  
          'bf'  : BF 
         } for i in range(len(loc))]

    with ThreadPool(num_threads) as tp:
        out = tp.map(getCube, di)
        print(ch) ##DEBUG
        sys.stdout.flush() #DEBUG

    ChanList.append(np.asarray(out))

outArray = np.asarray(ChanList)
## outArray will be a numpy array in [channel, synapse, z, y, x] order
## this is C-ordering


[222, 183, 24]
[191, 156, 5]
[220, 160, 42]
[197, 213, 24]
[215, 174, 40]
[188, 166, 12]
[220, 178, 28][195, 212, 28]

[220, 170, 42][190, 181, 22]

[195, 218, 25]
[225, 178, 14]
[188, 155, 40]
[203, 178, 32]
[206, 216, 24]
[234, 183, 10]
[189, 168, 32]
[200, 189, 42]
[210, 212, 18]
[235, 178, 14]
[188, 171, 40]
[220, 213, 18]
[215, 182, 36]
[235, 189, 12]
[191, 190, 32]
[218, 216, 22]
[222, 178, 34]
[237, 191, 13]
[190, 183, 43]
[220, 211, 30]
[221, 184, 34]
[236, 188, 15]
[196, 155, 6]
[228, 193, 5]
[223, 189, 34][252, 176, 6]

[231, 195, 10]
[222, 155, 14]
[255, 178, 6][218, 184, 37]

[227, 207, 12]
[197, 157, 16]
[222, 181, 41]
[235, 167, 16]
[201, 156, 23]
[244, 203, 14]
[237, 166, 34]
[227, 171, 18]
[251, 207, 10]
[205, 158, 28]
[234, 174, 20]
[226, 169, 32]
[237, 212, 5]
[219, 156, 26]
[227, 162, 26]
[225, 172, 36]
[227, 215, 12]
[236, 159, 14]
[230, 164, 28]
[231, 174, 38]
[250, 223, 5]
[242, 157, 14]
[232, 162, 26]
[237, 161, 40]
[249, 221, 11]
[229, 156, 20]
[232, 166, 26]
[2

[285, 211, 10]
[293, 249, 8]
[299, 209, 30]
[310, 218, 33]
[295, 221, 24]
[290, 255, 14]
[309, 217, 36]
[293, 220, 27][312, 245, 8]

[281, 220, 12]
[310, 223, 37]
[313, 220, 42]
[311, 253, 14]
[270, 203, 18][308, 208, 18]

[314, 222, 42]
[295, 224, 18]
[304, 223, 18]
[257, 197, 28]
[257, 236, 32]
[291, 226, 30]
[311, 216, 21]
[256, 206, 28]
[260, 229, 42]
[298, 225, 28]
[308, 208, 30]
[279, 192, 18]
[266, 237, 43]
[314, 210, 24]
[276, 193, 21]
[312, 226, 18]
[265, 240, 34]
[317, 219, 26]
[285, 202, 18]
[307, 226, 25]
[271, 247, 32]
[315, 221, 30]
[274, 194, 28]
[309, 227, 30]
[265, 250, 38]
[314, 245, 34]
[283, 196, 30]
[306, 233, 30]
[274, 243, 37]
[315, 241, 38]
[268, 214, 22]
[311, 233, 31]
[276, 245, 37]
[190, 268, 19]
[312, 237, 24]
[256, 218, 18]
[275, 251, 39]
[190, 274, 17]
[240, 258, 20]
[288, 245, 16]
[277, 251, 37]
[191, 274, 30][252, 262, 19]

[290, 242, 22]
[281, 249, 42]
[255, 280, 38]
[242, 261, 27][188, 279, 32]

[289, 239, 32]
[251, 256, 24]
[258, 260, 6]
[300, 232, 36

[284, 160, 12]
[305, 163, 20]
[253, 225, 10]
[255, 212, 35]
[262, 177, 8]
[307, 173, 29]
[255, 227, 14]
[243, 210, 43]
[275, 186, 6]
[314, 172, 28]
[250, 212, 40]
[254, 235, 15]
[259, 174, 17]
[292, 179, 18]
[252, 213, 43]
[227, 249, 14]
[260, 170, 21]
[192, 231, 37]
[235, 254, 10]
[289, 182, 16]
[268, 172, 20]
[238, 255, 8]
[200, 227, 38]
[290, 187, 16]
[265, 161, 29]
[240, 241, 10]
[207, 225, 38][296, 183, 26]

[258, 171, 28]
[255, 241, 14]
[195, 232, 34]
[308, 185, 16]
[283, 161, 18]
[244, 250, 11]
[308, 189, 21]
[208, 228, 34]
[282, 165, 23]
[255, 249, 10]
[312, 190, 28]
[210, 232, 34]
[282, 166, 22]
[226, 226, 20]
[277, 155, 32]
[210, 238, 36]
[286, 165, 20]
[239, 225, 20]
[278, 158, 34]
[222, 235, 36]
[282, 175, 22]
[231, 238, 30]
[284, 155, 36]
[217, 239, 38]
[284, 162, 26]
[233, 236, 26]
[277, 157, 42]
[200, 251, 32]
[283, 162, 31]
[311, 159, 35][249, 228, 18]

[200, 240, 42]
[278, 173, 30]
[245, 231, 25]
[315, 159, 34]
[210, 247, 35]
[258, 179, 16]
[254, 224, 24]
[304, 155, 40

[222, 178, 34]
[191, 190, 32]
[237, 191, 13]
[221, 184, 34]
[220, 211, 30]
[190, 183, 43]
[236, 188, 15]
[228, 193, 5]
[252, 176, 6][196, 155, 6]

[223, 189, 34]
[231, 195, 10]
[255, 178, 6]
[218, 184, 37]
[222, 155, 14]
[227, 207, 12]
[222, 181, 41]
[235, 167, 16]
[197, 157, 16]
[244, 203, 14]
[237, 166, 34]
[201, 156, 23]
[227, 171, 18]
[251, 207, 10]
[226, 169, 32]
[205, 158, 28]
[234, 174, 20]
[237, 212, 5]
[219, 156, 26]
[225, 172, 36]
[227, 162, 26]
[231, 174, 38]
[236, 159, 14]
[227, 215, 12]
[230, 164, 28]
[237, 161, 40]
[250, 223, 5]
[242, 157, 14]
[232, 162, 26]
[252, 171, 32]
[249, 221, 11][229, 156, 20]

[232, 166, 26]
[247, 167, 42]
[233, 160, 30]
[238, 195, 18]
[233, 154, 31]
[230, 177, 33]
[227, 171, 30]
[236, 196, 16]
[251, 159, 30]
[228, 183, 39]
[231, 195, 25]
[251, 165, 20]
[192, 162, 7]
[225, 176, 40]
[236, 199, 24]
[241, 169, 16]
[194, 171, 14]
[234, 180, 42]
[242, 196, 18]
[244, 174, 16]
[206, 174, 14]
[251, 179, 32]
[249, 195, 20]
[210, 167, 6]
[245, 170, 28]
[24

[308, 208, 18]
[291, 226, 30]
[276, 193, 21]
[274, 243, 37]
[304, 223, 18]
[285, 202, 18]
[276, 245, 37]
[311, 216, 21]
[274, 194, 28]
[298, 225, 28]
[275, 251, 39]
[308, 208, 30]
[283, 196, 30]
[312, 226, 18]
[314, 210, 24]
[277, 251, 37]
[268, 214, 22]
[307, 226, 25][317, 219, 26][256, 218, 18][281, 249, 42]



[315, 221, 30]
[314, 245, 34]
[309, 227, 30][289, 239, 32]

[240, 258, 20]
[315, 241, 38]
[306, 233, 30]
[300, 232, 36]
[252, 262, 19]
[303, 239, 37]
[190, 268, 19]
[311, 233, 31]
[242, 261, 27]
[292, 228, 40]
[312, 237, 24]
[190, 274, 17]
[251, 256, 24]
[292, 232, 42]
[288, 245, 16]
[191, 274, 30]
[240, 269, 26]
[311, 231, 38]
[290, 242, 22]
[188, 279, 32]
[315, 230, 40]
[231, 273, 17]
[255, 280, 38]
[191, 285, 38]
[258, 260, 6]
[224, 277, 20]
[299, 240, 35]
[193, 256, 12]
[275, 261, 7]
[235, 277, 17]
[303, 251, 34]
[198, 266, 9]
[285, 258, 7]
[224, 285, 22]
[217, 269, 6]
[302, 259, 20]
[274, 258, 9]
[228, 285, 20]
[292, 263, 28]
[220, 261, 14]
[283, 268, 10]
[233, 281, 16]
[

...

[271, 278, 8]
[198, 266, 9]
[313, 256, 18]
[231, 282, 24]
[263, 284, 12]
[217, 269, 6]
[317, 264, 20]
[247, 283, 16]
[279, 275, 12]
[220, 261, 14][309, 262, 29]

[244, 279, 24]
[285, 276, 14]
[250, 278, 26]
[218, 268, 12]
[300, 280, 17]
[286, 278, 14]
[194, 275, 6]
[253, 275, 30]
[292, 281, 24]
[275, 281, 13]
[200, 276, 15]
[292, 286, 25]
[255, 283, 27]
[298, 284, 24][280, 284, 12]

[198, 283, 8]
[249, 284, 31]
[261, 257, 21]
[213, 284, 8][310, 275, 17]

[197, 267, 32]
[267, 260, 17]
[219, 286, 8]
[310, 272, 19]
[207, 270, 34]
[194, 262, 22]
[264, 256, 20]
[262, 262, 36][203, 264, 38]

[267, 263, 21]
[194, 266, 23]
[205, 263, 42]
[261, 269, 32]
[203, 269, 17]
[258, 268, 18]
[265, 269, 39]
[210, 263, 34]
[200, 269, 22]
[258, 262, 42]
[221, 258, 37]
[263, 259, 30]
[272, 260, 37]
[212, 265, 38]
[266, 257, 26]
[201, 258, 28]
[285, 260, 32]
[218, 265, 37]
[221, 259, 16]
[268, 256, 30]
[217, 265, 40]
[273, 267, 34]
[220, 261, 22]
[257, 270, 30]
[276, 263, 40]
[219, 266, 42]
[220, 271, 21]
[2

[272, 166, 32]
[239, 240, 33]
[264, 177, 28]
[200, 198, 34]
[276, 174, 32]
[236, 244, 36]
[204, 193, 36]
[276, 183, 16]
[279, 175, 38]
[226, 252, 38]
[275, 178, 22]
[192, 205, 36]
[282, 171, 34]
[234, 240, 40]
[275, 183, 21]
[198, 197, 42]
[287, 165, 42]
[229, 248, 40]
[279, 180, 22]
[222, 198, 38][262, 214, 26][274, 173, 42][279, 185, 36]



[264, 231, 7]
[261, 191, 40]
[276, 190, 36]
[263, 212, 30]
[272, 182, 38]
[287, 188, 42]
[260, 230, 11]
[266, 221, 29]
[276, 184, 37]
[294, 161, 32]
[272, 208, 22]
[267, 227, 11]
[298, 255, 16]
[285, 214, 23]
[271, 228, 8]
[293, 166, 36]
[287, 233, 6]
[294, 243, 28]
[278, 217, 18]
[301, 163, 32]
[285, 224, 8]
[293, 249, 27]
[286, 212, 31]
[302, 167, 35]
[284, 231, 10]
[302, 248, 24]
[282, 216, 29]
[294, 170, 36]
[267, 254, 6]
[296, 250, 30]
[291, 195, 9]
[295, 172, 36]
[309, 254, 16]
[260, 244, 8]
[289, 195, 12]
[298, 171, 39]
[310, 248, 26]
[296, 162, 42][256, 242, 14]

[293, 205, 15]
[300, 164, 42]
[298, 200, 12]
[258, 198, 32]
[260, 247, 12]
[3

[191, 207, 26]
[203, 188, 12]
[224, 223, 16]
[194, 158, 34][208, 180, 6]

[189, 208, 16]
[194, 154, 38]
[229, 220, 18]
[213, 184, 13]
[187, 192, 34]
[198, 154, 38]
[231, 216, 23]
[217, 189, 14]
[190, 217, 36]
[231, 209, 26]
[195, 156, 36]
[201, 163, 19]
[188, 255, 32]
[237, 217, 26]
[205, 155, 32]
[188, 246, 40]
[200, 174, 18]
[246, 212, 18]
[204, 159, 42]
[190, 253, 42]
[202, 170, 23]
[242, 215, 26]
[194, 195, 8][206, 172, 22]

[211, 158, 32]
[202, 194, 8]
[217, 167, 19]
[219, 158, 33]
[205, 196, 8]
[223, 162, 22]
[221, 154, 40]
[250, 208, 24]
[211, 169, 21]
[203, 194, 12]
[250, 215, 24]
[228, 154, 36]
[214, 168, 24]
[199, 205, 12]
[248, 219, 24]
[235, 156, 32]
[194, 182, 18]
[251, 220, 24]
[254, 154, 38][201, 211, 12]

[206, 179, 20]
[195, 226, 8]
[195, 167, 38]
[205, 216, 12]
[192, 188, 22]
[204, 232, 12]
[205, 161, 32]
[215, 213, 12]
[223, 227, 9][200, 186, 16]

[197, 170, 36]
[197, 194, 18]
[210, 233, 8]
[205, 188, 16]
[193, 160, 42]
[197, 194, 20]
[200, 187, 22]
[212, 237, 11]
[2

[280, 284, 12]
[213, 284, 8]
[253, 275, 30]
[298, 284, 24]
[261, 257, 21]
[219, 286, 8]
[255, 283, 27]
[310, 275, 17]
[267, 260, 17]
[194, 262, 22]
[310, 272, 19]
[249, 284, 31]
[264, 256, 20]
[194, 266, 23]
[197, 267, 32]
[262, 262, 36]
[267, 263, 21]
[203, 269, 17]
[207, 270, 34][261, 269, 32]

[258, 268, 18]
[200, 269, 22]
[203, 264, 38]
[265, 269, 39]
[263, 259, 30]
[201, 258, 28]
[205, 263, 42]
[258, 262, 42]
[266, 257, 26]
[221, 259, 16]
[272, 260, 37]
[210, 263, 34]
[268, 256, 30]
[220, 261, 22]
[221, 258, 37]
[285, 260, 32]
[257, 270, 30]
[220, 271, 21]
[212, 265, 38]
[273, 267, 34]
[221, 268, 22]
[269, 265, 28]
[218, 265, 37]
[276, 263, 40]
[211, 263, 27]
[283, 260, 19]
[217, 265, 40]
[258, 282, 35]
[280, 258, 22]
[223, 262, 28]
[219, 266, 42]
[257, 283, 38]
[280, 269, 20]
[221, 267, 28]
[196, 274, 38]
[257, 282, 42]
[281, 260, 26]
[195, 276, 18]
[201, 273, 32]
[263, 285, 41]
[272, 266, 29]
[200, 273, 23]
[203, 275, 34]
[275, 273, 34]
[269, 277, 18][194, 283, 20]

[203, 284, 3

[276, 190, 36]
[262, 214, 26]
[264, 231, 7]
[272, 182, 38]
[287, 188, 42]
[263, 212, 30]
[260, 230, 11][276, 184, 37]

[294, 161, 32]
[266, 221, 29]
[298, 255, 16]
[267, 227, 11]
[293, 166, 36]
[272, 208, 22]
[271, 228, 8]
[294, 243, 28]
[285, 214, 23]
[301, 163, 32]
[287, 233, 6]
[293, 249, 27]
[278, 217, 18]
[302, 167, 35]
[285, 224, 8]
[302, 248, 24]
[284, 231, 10]
[286, 212, 31]
[294, 170, 36]
[296, 250, 30]
[267, 254, 6]
[282, 216, 29]
[309, 254, 16]
[295, 172, 36]
[260, 244, 8]
[310, 248, 26][298, 171, 39]

[291, 195, 9]
[256, 242, 14]
[296, 162, 42]
[289, 195, 12][258, 198, 32]

[300, 164, 42]
[260, 247, 12]
[293, 205, 15]
[271, 198, 32]
[258, 251, 9]
[309, 175, 35]
[298, 200, 12]
[259, 202, 32]
[264, 250, 8]
[292, 177, 36]
[309, 192, 7]
[256, 202, 36]
[280, 249, 6]
[259, 205, 41]
[303, 179, 33]
[308, 193, 11]
[275, 243, 10]
[296, 178, 39][267, 204, 40]

[305, 196, 10]
[284, 244, 9]
[301, 209, 6]
[300, 178, 36]
[274, 196, 36]
[301, 209, 6]
[282, 241, 14]
[296, 182, 38]
[273, 195

In [55]:
outArray.shape

(12, 813)

In [11]:
#np.savetxt("rorb_avg_at.csv", np.column_stack([ids, outArray]), delimiter=",", fmt = "%d", header = "ids, ")
#np.concatenate((trmp, outArray), axis = 2)

In [12]:
#np.hstack((np.transpose(np.asarray(ids)),outArray))

In [13]:
if False:
    head = "DAPI1, DAPI2, DAPI3, GABA, GAD2, Gephyrin, GluN1, MBP, PSD95, synapsin, TdTomato, VGlut1"
    np.savetxt("rorb_gaussianAvg_at.csv", np.transpose(outArray), delimiter=",", fmt = "%f", header = head)


In [14]:
if False:
    np.savetxt("rorb_gaussianAvg_at_orderIDS.csv", np.transpose(ids), delimiter=",", fmt = "%d", header = 'ids')

In [15]:
if False:
    np.savetxt("rorb_gaussianAvg_at_orderLocations.csv", loc, delimiter=",", fmt = "%d", header = 'x,y,z')

In [56]:
if LOCATIONS == "GoogleDocData.csv":
    head = "DAPI1, DAPI2, DAPI3, GABA, GAD2, Gephyrin, GluN1, MBP, PSD95, synapsin, TdTomato, VGlut1"
    np.savetxt("rorb_gaussianAvg_at_GoogleDoc.csv", np.transpose(outArray), delimiter=",", fmt = "%f", header = head)